In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

In [12]:
directory = r"dataset"
categories = ["with_mask", "without_mask"]

data = []
labels = []

#reading images from the dateset directory
for category in categories:
    path = os.path.join(directory,category)
    for image in os.listdir(path):
        img_path = os.path.join(path, image)
        image = load_img(img_path, target_size = (224,224))
        image = img_to_array(image)
        image = preprocess_input(image)
        data.append(image)
        labels.append(category)        

/usr/lib/python3/dist-packages/PIL/Image.py:918: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


In [13]:
#one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data, dtype = "float32")
labels = np.array(labels)

trainX, testX, trainY, testY = train_test_split(data, labels, test_size=0.20, stratify=labels, random_state=42)

In [14]:
#augmentating the data
aug = ImageDataGenerator(rotation_range = 20,
                         zoom_range = 0.15,
                         width_shift_range = 0.2,
                         height_shift_range = 0.2,
                         shear_range = 0.15,
                         horizontal_flip = True,
                         fill_mode = "nearest")

In [15]:
# load the MobileNetV2 network, ensuring the head FC layer sets are off
basemodel = MobileNetV2(weights = "imagenet", include_top = False, input_tensor = Input(shape = (224,224,3)))

9412608/9406464 [==============================] - 36s 4us/step


In [18]:
# construct the head of the model that will be placed on top of the base model
headmodel = basemodel.output
headmodel = AveragePooling2D(pool_size = (7, 7))(headmodel)
headmodel = Flatten(name = 'flatten')(headmodel)
headmodel = Dense(128, activation = 'relu')(headmodel)
headmodel = Dense(2, activation = 'softmax')(headmodel)

# place the head FC model on top of the base model (this will become the actual model we will train)
model = Model(inputs = basemodel.input, outputs = headmodel)

In [19]:
#freeze all the basemodel layers so they will not be updated during training
for layer in basemodel.layers:
    layer.trainable = False

In [20]:
optimizer = Adam(lr = 1e-4, decay = 1e-4/20)
#compiling model
model.compile(loss = "binary_crossentropy", optimizer = optimizer, metrics = ["accuracy"])

In [22]:
#training headmodel
trained_model = model.fit(aug.flow(trainX, trainY, batch_size = 32),
                         steps_per_epoch = len(trainX)//32,
                         validation_data = (testX, testY),
                         validation_steps = len(testX)//32,
                         epochs = 20)

Epoch 1/20
95/95 [==============================] - 60s 633ms/step - loss: 0.2333 - accuracy: 0.9038 - val_loss: 0.0798 - val_accuracy: 0.9791
Epoch 2/20
95/95 [==============================] - 66s 691ms/step - loss: 0.0632 - accuracy: 0.9825 - val_loss: 0.0491 - val_accuracy: 0.9857
Epoch 3/20
95/95 [==============================] - 64s 676ms/step - loss: 0.0483 - accuracy: 0.9842 - val_loss: 0.0414 - val_accuracy: 0.9870
Epoch 4/20
95/95 [==============================] - 61s 643ms/step - loss: 0.0416 - accuracy: 0.9878 - val_loss: 0.0421 - val_accuracy: 0.9870
Epoch 5/20
95/95 [==============================] - 61s 641ms/step - loss: 0.0290 - accuracy: 0.9914 - val_loss: 0.0352 - val_accuracy: 0.9883
Epoch 6/20
95/95 [==============================] - 61s 639ms/step - loss: 0.0305 - accuracy: 0.9885 - val_loss: 0.0278 - val_accuracy: 0.9935
Epoch 7/20
95/95 [==============================] - 61s 641ms/step - loss: 0.0288 - accuracy: 0.9904 - val_loss: 0.0277 - val_accuracy: 0.9922

In [28]:
#prediction
predict = model.predict(testX, batch_size = 32)

In [29]:
#based on largeset probability between with mask and without mask we need to classify the images 
predict = np.argmax(predict, axis = 1)
#classificaiton report
print(classification_report(testY.argmax(axis = 1), predict, target_names = lb.classes_))

              precision    recall  f1-score   support

   with_mask       0.98      1.00      0.99       383
without_mask       1.00      0.98      0.99       384

    accuracy                           0.99       767
   macro avg       0.99      0.99      0.99       767
weighted avg       0.99      0.99      0.99       767



In [30]:
#saving model to the disk
model.save("mask_detector.model", save_format="h5")